In [1]:
import torch

In [40]:
scores = torch.rand(5,4)
targets=torch.rand(5,4)
print(scores)

tensor([[0.2472, 0.6547, 0.7341, 0.4455],
        [0.0959, 0.3835, 0.9753, 0.9571],
        [0.3276, 0.6565, 0.8749, 0.1919],
        [0.7725, 0.8318, 0.8900, 0.7010],
        [0.9623, 0.7471, 0.1597, 0.3891]])


In [51]:
def average_precision_per_class2(scores, targets:torch.Tensor):
    ap = torch.zeros(scores.size(1))
    rg = torch.arange(1, scores.size(0) + 1).float()
    # compute average precision for each class
    batch_sz=2
    for k in range((scores.size(1)-1)//batch_sz+1):
        # sort scores
        scores_k = scores[:, k*batch_sz:(k+1)*batch_sz]
        targets_k = targets[:, k*batch_sz:(k+1)*batch_sz]
        _, sortind = torch.sort(scores_k, 0, True)
        sortcol = torch.arange(0,sortind.shape[1]).repeat(sortind.shape[0],1)
        print(_)
        print(sortind)
        print(sortcol)
        print(sortind.shape)
        truth = targets_k[sortind,sortcol]
        score_truth = scores_k[sortind,sortcol]
        # print(score_truth)
        # print('')
        tp = truth.float().cumsum(1)
        # compute precision curve
        # print(truth.shape)
        # print(tp.shape)
        # print(rg.shape)
        # break
        precision = tp.div(rg.unsqueeze(1))
        # compute average precision
        ap[k*batch_sz:(k+1)*batch_sz] = precision[truth.bool()].sum(0) / max(float(truth.sum()), 1)
    return ap

In [ ]:
print(average_precision_per_class2(scores,targets))

In [103]:
def average_precision_per_class(scores, targets):
    ap = torch.zeros(scores.size(1))
    rg = torch.arange(1, scores.size(0) + 1).float()
    # compute average precision for each class
    for k in range(scores.size(1)):
        # sort scores
        scores_k = scores[:, k]
        targets_k = targets[:, k]
        _, sortind = torch.sort(scores_k, 0, True)
        truth = targets_k[sortind]
        tp = truth.float().cumsum(0)
        # compute precision curve
        precision = tp.div(rg)
        # compute average precision
        ap[k] = precision[truth.bool()].sum() / max(float(truth.sum()), 1)
        print("truth",truth,tp)
        print("ap",ap,precision[truth.bool()],max(float(truth.sum()), 1))
    return ap

In [105]:
def average_precision_per_class2(scores, targets:torch.Tensor):
    ap = torch.zeros(scores.size(1))
    rg = torch.arange(1, scores.size(0) + 1).float()
    # compute average precision for each class
    batch_sz=2
    for k in range((scores.size(1)-1)//batch_sz+1):
        # sort scores
        scores_k = scores[:, k*batch_sz:(k+1)*batch_sz]
        targets_k = targets[:, k*batch_sz:(k+1)*batch_sz]
        _, sortind = torch.sort(scores_k, 0, True)
        sortcol = torch.arange(0,sortind.shape[1]).repeat(sortind.shape[0],1)
        truth = targets_k[sortind,sortcol]
        tp = truth.float().cumsum(0)
        # compute precision curve
        precision = tp.div(rg.unsqueeze(1))
        # compute average precision
        ap[k*batch_sz:(k+1)*batch_sz] = precision[truth.bool()].view_as(truth).sum(0) / torch.clip((truth.sum(0)), min=1)
    return ap

In [106]:
print("correct",average_precision_per_class(scores,targets))
print("------------------------------")

print("mine",average_precision_per_class2(scores,targets))

truth tensor([0.3651, 0.0983, 0.8205, 0.0642, 0.4067]) tensor([0.3651, 0.4634, 1.2839, 1.3481, 1.7548])
ap tensor([0.9760, 0.0000, 0.0000, 0.0000]) tensor([0.3651, 0.2317, 0.4280, 0.3370, 0.3510]) 1.7548457384109497
truth tensor([0.1815, 0.6678, 0.8743, 0.6479, 0.0786]) tensor([0.1815, 0.8493, 1.7236, 2.3716, 2.4501])
ap tensor([0.9760, 0.9239, 0.0000, 0.0000]) tensor([0.1815, 0.4247, 0.5745, 0.5929, 0.4900]) 2.4501490592956543
truth tensor([0.1016, 0.0624, 0.7112, 0.4481, 0.0738]) tensor([0.1016, 0.1639, 0.8752, 1.3233, 1.3970])
ap tensor([0.9760, 0.9239, 0.7770, 0.0000]) tensor([0.1016, 0.0820, 0.2917, 0.3308, 0.2794]) 1.3970437049865723
truth tensor([0.7820, 0.8700, 0.1454, 0.5552, 0.2111]) tensor([0.7820, 1.6520, 1.7973, 2.3525, 2.5636])
ap tensor([0.9760, 0.9239, 0.7770, 1.2904]) tensor([0.7820, 0.8260, 0.5991, 0.5881, 0.5127]) 2.5635673999786377
correct tensor([0.9760, 0.9239, 0.7770, 1.2904])
------------------------------
ap tensor([0.9760, 0.9239, 0.0000, 0.0000]) tensor([[0.3

In [108]:
import torch.nn.functional as F
import torch
import numpy as np

def cross_entropy_loss(scores, targets:torch.Tensor):
    scores=F.softmax(scores.float(), dim=0)
    loss = 0
    for i in range(len(scores)):
        loss = loss + (-1 * targets[i]*np.log(scores[i]))
    return loss
 

cross_entropy_loss(scores, targets)

tensor([3.0151, 3.8960, 2.1993, 3.9154])